In [56]:
cd /opt/ml/code

/opt/ml/code


In [57]:
import random
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import segmentation_models_pytorch as smp
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR, ExponentialLR
from torch.optim.sgd import SGD
from adamp import AdamP

from warmup_scheduler import GradualWarmupScheduler
from utils_modified import label_accuracy_score, add_hist

import cv2

import numpy as np
import pandas as pd

from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
import timeit

print(f'pytorch version: {torch.__version__}')
print(f'GPU 사용 가능 여부: {torch.cuda.is_available()}')
print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

device = 'cuda' if torch.cuda.is_available() else 'cpu'

pytorch version: 1.7.1
GPU 사용 가능 여부: True
Tesla P40
1


In [58]:
def set_seed(seed=21):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    #torch.set_deterministic(True)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# 테스트 함수

In [59]:
def EDA():
    with open(anns_file_path, 'r') as f:
        dataset = json.loads(f.read())

    categories = dataset['categories']
    anns = dataset['annotations']
    imgs = dataset['images']
    nr_cats = len(categories)
    nr_annotations = len(anns)
    nr_images = len(imgs)

    cat_names = []
    super_cat_names = []
    super_cat_ids = {}
    super_cat_last_name = ''
    nr_super_cats = 0
    for cat_it in categories:
        cat_names.append(cat_it['name'])
        super_cat_name = cat_it['supercategory']
        if super_cat_name != super_cat_last_name:
            super_cat_names.append(super_cat_name)
            super_cat_ids[super_cat_name] = nr_super_cats
            nr_super_cats += 1
    print('Number of super categories:', nr_super_cats)
    print('Number of categories:', nr_cats)
    print('Number of annotations:', nr_annotations)
    print('Number of images:', nr_images)

    cat_histogram = np.zeros(nr_cats, dtype=int)
    for ann_it in anns:
        cat_histogram[ann_it['category_id']] += 1

    f, ax = plt.subplots(figsize=(5, 5))
    df = pd.DataFrame({'Categories': cat_names, 'Number of annotations':cat_histogram})

    plt.title('category distribution of train set')
    sns.barplot(x='Number of annotations', y='Categories', data=df.sort_values('Number of annotations', ascending=False), label='Total', color='b')
    plt.show()

In [60]:
def test_dataloader(dataloader, idx=0):
    data = iter(dataloader).next()
    if len(data) == 3:
        imgs, masks, image_infos = data
        img = imgs[idx]
        mask = masks[idx]
        image_info = image_infos[idx]

        fig, axes = plt.subplots(1, 2, figsize=(12, 12))
        print('image shape:', list(img.shape))
        print('mask shape:', list(mask.shape))
        print('Unique values, category of transformed mask:\n', {int(i):category_names[int(i)] for i in list(np.unique(mask))})

        axes[0].imshow((np.array(img.permute([1, 2, 0]))).astype(np.uint8))
        axes[0].grid(False)
        axes[0].set_title('imput image:' + str(image_info['file_name']), fontsize=15)

        axes[1].imshow(mask)
        axes[1].grid(False)
        axes[1].set_title('masks :' + str(image_info['file_name']), fontsize=15)

        plt.show()
    elif len(data) == 2:
        imgs, image_infos = iter(dataloader).next()
        img = imgs[0]
        image_info = image_infos[0]

        fig, ax = plt.subplots(figsize=(6, 6))
        print('image shape:', list(img.shape))

        ax.imshow(img.permute([1, 2, 0]))
        ax.grid(False)
        ax.set_title('input image:' + str(image_info['file_name']), fontsize=15)

        plt.show()

In [61]:
def test_eval_model(model, dataloader, idx=0):
    for imgs, image_infos in dataloader:
        model.eval()
        outs = model(torch.stack(imgs).to(device))
        outs = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
        break

    imgs = imgs[idx]
    image_infos = image_infos[idx]
    outs = outs[idx]

    fig, axes = plt.subplots(1, 2, figsize=(16, 16))
    print('Shape of Original Image:', list(imgs.shape))
    print('Shape of Predicted:', list(outs.shape))
    print('Unique values, category of transformed mask\n', {int(i):category_names[int(i)] for i in list(np.unique(outs))})

    axes[0].imshow(imgs.permute([1,2,0]))
    axes[0].grid(False)
    axes[0].set_title('Original image:' + str(image_infos['file_name']), fontsize=15)

    axes[1].imshow(outs)
    axes[1].grid(False)
    axes[1].set_title('Predicted:' + str(image_infos['file_name']), fontsize=15)

    plt.show()

# 데이터 셋 / 로더

In [62]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, transform):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        self.aug = A.Compose([
            A.HorizontalFlip(),
            A.VerticalFlip(),
            A.RandomRotate90(),
            A.OneOf([
                A.MotionBlur(p=1.0),
                A.OpticalDistortion(p=1.0)
            ], p=2/3),
        ])
    
    def __getitem__(self, index:int):
        image_infos = self.coco.loadImgs(index)[0]

        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        #images /= 255.0

        if self.mode in ('train', 'val'):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            masks = np.zeros((image_infos['height'], image_infos['width']))
            for ann in anns:
                masks = np.maximum(self.coco.annToMask(ann) * (ann['category_id'] + 1), masks)
            
            images, masks = A.Resize(image_size, image_size)(image=images, mask=masks).values()
            images_aug, masks_aug = self.aug(image=images, mask=masks).values()
            images, masks = self.transform(image=images, mask=masks).values()
            images_aug, masks_aug = self.transform(image=images_aug, mask=masks_aug).values()
            return ((images, masks, image_infos), (images_aug, masks_aug, image_infos))
        
        if self.mode == 'test':
            images = self.transform(image=images)['image']
            return ((images, image_infos), )

    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [63]:
def get_dataloader():
    train_dataset = CustomDataset(data_dir=train_path, mode='train',transform=train_transform)
    val_dataset = CustomDataset(data_dir=val_path, mode='val', transform=val_transform)
    test_dataset = CustomDataset(data_dir=test_path, mode='test', transform=test_transform)

    def collate_fn(x):
        x = np.array(x).transpose(2, 0, 1)
        a, b, c = x.shape
        x = np.reshape(x, (a, b * c))
        return x.tolist()

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=batch_size,
                                            shuffle=True,
                                            num_workers=4,
                                            collate_fn=collate_fn,
                                            drop_last=True,
                                            worker_init_fn=seed_worker)

    val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                            batch_size=batch_size,
                                            shuffle=False,
                                            num_workers=4,
                                            collate_fn=collate_fn,
                                            worker_init_fn=seed_worker)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=batch_size,
                                            num_workers=4,
                                            collate_fn=collate_fn,
                                            worker_init_fn=seed_worker)
    return train_loader, val_loader, test_loader

In [64]:
def get_folded_dataloader(kfold=5):
    train_dataset = CustomDataset(data_dir=train_all_path, mode='train',transform=train_transform)

    train_all_size = len(train_dataset)
    size_list = [train_all_size // kfold] * kfold
    size_list = [train_all_size // kfold] * kfold
    for i in range(train_all_size % kfold):
        size_list[i] += 1
    train_dataset_list = torch.utils.data.random_split(train_dataset, size_list)

    def collate_fn(batch):
        return tuple(zip(*batch))

    for k in range(kfold):
        train_loader = torch.utils.data.ConcatDataset(train_dataset_list[:k] + train_dataset_list[k + 1:])
        val_loader = train_dataset_list[k]

        train_loader = torch.utils.data.DataLoader(dataset=train_loader, 
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    num_workers=4,
                                                    collate_fn=collate_fn,
                                                    drop_last=True,
                                                    worker_init_fn=seed_worker)
        
        val_loader = torch.utils.data.DataLoader(dataset=val_loader, 
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    num_workers=4,
                                                    collate_fn=collate_fn,
                                                    drop_last=True,
                                                    worker_init_fn=seed_worker)

        yield train_loader, val_loader

# 학습 / 추론

In [65]:
def train(identifier, num_epochs, model, data_loader, val_loader, criterion, optimizer, scheduler, saved_dir, val_every=1, device='cuda', print_log=True):
    if print_log:
        print('Start training')
    best_mIoU = 0
    hist_mIoU = []
    epoch_begin = 0

    name = str(type(model.encoder)).split('.')[-1]
    name += ' ' + str(type(model.decoder)).split('.')[-1]

    for epoch in tqdm(range(0, num_epochs), desc=name):
        if scheduler is not None:
            scheduler.step(epoch)
        if print_log:
            print(f"optimizer{optimizer.param_groups[0]['lr']}")
        model.train()
        for step, (images, masks, _) in tqdm(enumerate(data_loader), desc='Training', leave=False, total=len(data_loader)):
            images = torch.stack(images).to(device)
            masks = torch.stack(masks).long().to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            if print_log:
                if (step + 1) % 25 == 0:
                    print(f'Epoch[{epoch + 1}/{num_epochs}], Step[{step + 1}/{len(data_loader)}], Loss: {loss.item():.4f}')
                    
        if (epoch + 1) % val_every == 0:
            mIoU = validation(epoch + 1, model, val_loader, criterion, device, print_log)
            hist_mIoU.append(mIoU)
            if mIoU > best_mIoU:
                if print_log:
                    print(f'Best performance at epoch: {epoch + 1}')
                    print('Save model in', saved_dir)
                best_mIoU = mIoU
                save_model(identifier, model, saved_dir)

    return hist_mIoU

In [66]:
def validation(epoch, model, data_loader, criterion, device, print_log=True):
    if print_log:
        print(f'Start validation #{epoch}')
    model.eval()
    with torch.no_grad():
        hist = np.zeros((N_CLASSES, N_CLASSES))
        for step, (images, masks, _) in tqdm(enumerate(data_loader), desc='Validation', leave=False, total=len(data_loader)):
            images = torch.stack(images).to(device)
            masks = torch.stack(masks).long().to(device)
            outputs = model(images)

            loss = criterion(outputs, masks)

            outputs = torch.argmax(outputs.squeeze(), dim=1)
            outputs = outputs.detach().cpu().numpy()
            masks = masks.detach().cpu().numpy()
            hist = add_hist(hist, masks, outputs, n_class=N_CLASSES)
        
        acc, acc_cls, mIoU, fwavacc = label_accuracy_score(hist)
        if print_log:
            print(f'Validation #{epoch} mIoU: {mIoU:.4f}')
    return mIoU

In [67]:
def save_model(identifier, model, saved_path):
    file_name = f'{identifier}.pt'
    os.makedirs(saved_path, exist_ok=True)
    check_point = {'net':model.state_dict()}
    output_path = os.path.join(saved_path, file_name)
    torch.save(model.state_dict(), output_path)

In [68]:
def load_model(identifier, model, saved_path):
    load_dir = saved_path + f'/{identifier}.pt'
    checkpoint = torch.load(load_dir, map_location=device)
    model.load_state_dict(checkpoint)
    return model

In [69]:
def test(model, data_loader, device):
    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction')
    print(model, type(model))
    model.eval()

    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    
    with torch.no_grad():
        for step, (imgs, image_infos) in tqdm(enumerate(data_loader), total=len(data_loader)):
            outs = model(torch.stack(imgs).to(device))
            outs = torch.argmax(outs, dim=1).detach().cpu().numpy()

            masks = []
            for img, mask in zip(np.stack(imgs), outs):
                img, mask = transform(image=img, mask=mask).values()
                masks.append(mask)
            outs = np.array(masks)
            outs = outs.reshape([outs.shape[0], size * size]).astype(int)
            preds_array = np.vstack((preds_array, outs))

            file_name_list.append([i['file_name'] for i in image_infos])
    print('End prediction.')
    file_names = [y for x in file_name_list for y in x]
    return file_names, preds_array

In [70]:
def make_submission(identifier, model, dataloader):
    submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)
    file_names, preds = test(model, dataloader, device)

    for file_name, string in zip(file_names, preds):
        submission = submission.append({'image_id':file_name, 'PredictionString':' '.join(str(e) for e in string.tolist())}, ignore_index=True)

    submission.to_csv(submission_path + f'/{identifier}.csv', index=False)

In [71]:
def model_test(encoder_list, decoder_list):
    model_test_result = []
    for encoder in encoder_list:
        for  decoder in decoder_list:
            set_seed(random_seed)
            try:
                if decoder == 'DeepLabV3Plus':
                    model = smp.DeepLabV3Plus(
                        encoder_name=encoder,
                        encoder_weights="imagenet",
                        in_channels=3,
                        classes=N_CLASSES,
                    )
                else:
                    raise 'decoder does not exist'
                criterion = nn.CrossEntropyLoss()
                optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=1e-6)
                model = model.to(device)
                start_time = timeit.default_timer()
                mIoU = train(num_epochs, model, train_loader, val_loader, criterion, optimizer, saved_path, val_every, device, print_log=False)
                end_time = timeit.default_timer()
                time = end_time - start_time
                stat = {'encoder' : encoder, 'decoder' : decoder, 'mIoU' : mIoU, 'time' : time}
            except Exception as e:
                stat = {'encoder' : encoder, 'decoder' : decoder, 'error' : e}
                pass
            print(stat)
            model_test_result.append(stat)
    return model_test_result

In [72]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=12, smoothing=0.1, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))
    
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )
    
class CustomLoss():
    def __init__(self, w1, w2, device):
        self.w1 = w1
        self.w2 = w2
        self.CrossEntropyLoss = nn.CrossEntropyLoss().to(device)
        self.FocalLoss = FocalLoss().to(device)
    
    def __call__(self, predict, y):
        loss1 = self.CrossEntropyLoss(predict, y)
        loss2 = self.FocalLoss(predict, y)
        return loss1 * self.w1 + loss2 * self.w2

In [73]:
class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

# 하이퍼파라미터

In [75]:
N_CLASSES = 12
batch_size = 8
num_epochs = 60
image_size = 256
learning_rate = 0.00005
identifier = 'resnext101_32x4d_aug2_loss_warmup_Adamp'
random_seed = 21
val_every = 1
warmup_epo = 1

set_seed(random_seed)

model = smp.DeepLabV3Plus(
    encoder_name="efficientnet-b4",
    encoder_weights="imagenet",
    in_channels=3,
    classes=N_CLASSES,
).to(device)
criterion = CustomLoss(0.9, 0.1, device)
optimizer = AdamP(model.parameters(), lr=learning_rate, weight_decay=1e-6)
scheduler_reduce = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=3, total_epoch=warmup_epo, after_scheduler=scheduler_reduce)


train_transform = A.Compose([
    #A.Resize(image_size, image_size),
    A.Normalize(),
    ToTensorV2()
])

val_transform = A.Compose([
    #A.Resize(image_size, image_size),
    A.Normalize(),
    ToTensorV2()
])

test_transform = A.Compose([
    #A.Resize(image_size, image_size),
    A.Normalize(),
    ToTensorV2()
])

dataset_path = '../input/data'
anns_file_path = dataset_path + '/train.json'
train_path = dataset_path + '/train.json'
train_all_path = dataset_path + '/train_all.json'
val_path = dataset_path + '/val.json'
test_path = dataset_path + '/test.json'
saved_path = './saved'
submission_path = './submission'
category_names = ['Background','UNKNOWN','General trash','Paper','Paper pack','Metal','Glass','Plastic','Styrofoam','Plastic bag','Battery','Clothing']


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /opt/ml/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth


  0%|          | 0.00/74.4M [00:00<?, ?B/s]

In [76]:
set_seed(random_seed)
train_loader, val_loader, test_loader = get_dataloader()

loading annotations into memory...
Done (t=4.63s)
creating index...
index created!
loading annotations into memory...
Done (t=0.85s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [77]:
set_seed(random_seed)
train(identifier, num_epochs, model, train_loader, val_loader, criterion, optimizer, scheduler_warmup, saved_path, print_log=True)

Start training


EfficientNetEncoder'> DeepLabV3PlusDecoder'>:   0%|          | 0/60 [00:00<?, ?it/s]

optimizer0.00015000000000000001


Training:   0%|          | 0/327 [00:00<?, ?it/s]

Epoch[1/60], Step[25/327], Loss: 2.0917
Epoch[1/60], Step[50/327], Loss: 1.4514
Epoch[1/60], Step[75/327], Loss: 1.4459
Epoch[1/60], Step[100/327], Loss: 1.2492
Epoch[1/60], Step[125/327], Loss: 1.1802
Epoch[1/60], Step[150/327], Loss: 1.2347
Epoch[1/60], Step[175/327], Loss: 1.0728
Epoch[1/60], Step[200/327], Loss: 1.0562
Epoch[1/60], Step[225/327], Loss: 1.1610
Epoch[1/60], Step[250/327], Loss: 0.8868
Epoch[1/60], Step[275/327], Loss: 1.0375
Epoch[1/60], Step[300/327], Loss: 0.8986
Epoch[1/60], Step[325/327], Loss: 0.8976
Start validation #1


...

Training:   0%|          | 0/327 [00:00<?, ?it/s]

Epoch[2/60], Step[25/327], Loss: 0.8396
Epoch[2/60], Step[50/327], Loss: 0.9747
Epoch[2/60], Step[75/327], Loss: 1.2300
Epoch[2/60], Step[100/327], Loss: 1.2344
Epoch[2/60], Step[125/327], Loss: 0.9183
Epoch[2/60], Step[150/327], Loss: 0.9359
Epoch[2/60], Step[175/327], Loss: 0.8659
Epoch[2/60], Step[200/327], Loss: 0.8432
Epoch[2/60], Step[225/327], Loss: 0.9545
Epoch[2/60], Step[250/327], Loss: 0.8572
Epoch[2/60], Step[275/327], Loss: 0.9523
Epoch[2/60], Step[300/327], Loss: 0.8928
Epoch[2/60], Step[325/327], Loss: 1.3355
Start validation #2


Validation:   0%|          | 0/82 [00:00<?, ?it/s]

Validation #2 mIoU: 0.3817
Best performance at epoch: 2
Save model in ./saved


TypeError: float() argument must be a string or a number, not 'NoneType'

### 임시 코드

### kfold

In [61]:
kfold = 5
for k, (train_loader, val_loader) in enumerate(get_folded_dataloader(kfold)):
    identifier = f'timm-regnetx_064_update'
    model = smp.DeepLabV3Plus(
        encoder_name="timm-regnetx_064",
        encoder_weights="imagenet",
        in_channels=3,
        classes=N_CLASSES,
    ).to(device)
    criterion = CustomLoss(0.9, 0.1, device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=1e-6)
    scheduler_steplr = StepLR(optimizer, step_size=10, gamma=0.1)
    scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=5, after_scheduler=scheduler_steplr)

    train(identifier, num_epochs, model, train_loader, val_loader, criterion, optimizer, scheduler_warmup, saved_path, print_log=False)

loading annotations into memory...
Done (t=6.16s)
creating index...
index created!


RegNetEncoder'> DeepLabV3PlusDecoder'>:   0%|          | 0/40 [00:00<?, ?it/s]

0 0.0


Training:   0%|          | 0/81 [00:00<?, ?it/s]

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataset.py", line 218, in __getitem__
    return self.datasets[dataset_idx][sample_idx]
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataset.py", line 272, in __getitem__
    return self.dataset[self.indices[idx]]
  File "<ipython-input-44-76439f55ae0b>", line 39, in __getitem__
    images_aug, masks_aug = self.transform(image=images, 

### ensamble

In [ ]:
ensamble_path = [
    '/opt/ml/code/saved/timm-regnetx_064_0-5fold.pt',
    '/opt/ml/code/saved/timm-regnetx_064_1-5fold.pt',
    '/opt/ml/code/saved/timm-regnetx_064_2-5fold.pt',
    '/opt/ml/code/saved/timm-regnetx_064_3-5fold.pt',
    '/opt/ml/code/saved/timm-regnetx_064_4-5fold.pt'
]
model_list = []
for p in ensamble_path:
    model_path = p
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint)
    model_list.append(model)
make_submission(model_list, test_loader)

In [24]:
set_seed(random_seed)
identifier = f'resnext101_32x4d_aug_loss_AdamP_warmup'
model = smp.DeepLabV3Plus(
    encoder_name="resnext101_32x4d",
    encoder_weights="imagenet",
    in_channels=3,
    classes=N_CLASSES,
).to(device)
criterion = CustomLoss(0.9, 0.1, device)
optimizer = AdamP(model.parameters(), lr=learning_rate)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs-warmup_epo)
scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)

train(identifier, num_epochs, model, train_loader, val_loader, criterion, optimizer, scheduler_warmup, saved_path, print_log=True)

Start training


RegNetEncoder'> DeepLabV3PlusDecoder'>:   0%|          | 0/60 [00:00<?, ?it/s]

optimizer5e-05
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use ca

Training:   0%|          | 0/327 [00:00<?, ?it/s]

Epoch[1/60], Step[25/327], Loss: 2.2118
Epoch[1/60], Step[50/327], Loss: 1.8856
Epoch[1/60], Step[75/327], Loss: 1.6470
Epoch[1/60], Step[100/327], Loss: 1.4770
Epoch[1/60], Step[125/327], Loss: 1.6162
Epoch[1/60], Step[150/327], Loss: 1.4114
Epoch[1/60], Step[175/327], Loss: 1.3691
Epoch[1/60], Step[200/327], Loss: 1.2613
Epoch[1/60], Step[225/327], Loss: 1.2535
Epoch[1/60], Step[250/327], Loss: 1.1376
Epoch[1/60], Step[275/327], Loss: 1.0576
Epoch[1/60], Step[300/327], Loss: 1.1279
Epoch[1/60], Step[325/327], Loss: 1.0544
Start validation #1


...

Training:   0%|          | 0/327 [00:00<?, ?it/s]

Epoch[60/60], Step[25/327], Loss: 0.5481
Epoch[60/60], Step[50/327], Loss: 0.6052
Epoch[60/60], Step[75/327], Loss: 0.5686
Epoch[60/60], Step[100/327], Loss: 0.5676
Epoch[60/60], Step[125/327], Loss: 0.5634
Epoch[60/60], Step[150/327], Loss: 0.5454
Epoch[60/60], Step[175/327], Loss: 0.5458
Epoch[60/60], Step[200/327], Loss: 0.5817
Epoch[60/60], Step[225/327], Loss: 0.5615
Epoch[60/60], Step[250/327], Loss: 0.5687
Epoch[60/60], Step[275/327], Loss: 0.5963
Epoch[60/60], Step[300/327], Loss: 0.5507
Epoch[60/60], Step[325/327], Loss: 0.5649
Start validation #60


Validation:   0%|          | 0/82 [00:00<?, ?it/s]

Validation #60 mIoU: 0.4978


[0.2658736133065894,
 0.2721227170859234,
 0.3086709786480146,
 0.32119438042777315,
 0.36396721504179025,
 0.35420742416880735,
 0.3520882886678196,
 0.387331879061339,
 0.33015373318692226,
 0.3519429395864824,
 0.34395400280193916,
 0.38811108891058227,
 0.3789410675916069,
 0.3862952121751992,
 0.3896647157526525,
 0.3863651595672491,
 0.39118501444827464,
 0.44428706560790404,
 0.389875084758762,
 0.38303400904015694,
 0.4290018042119155,
 0.42504658996109534,
 0.43326107253030366,
 0.4421288882503685,
 0.4435767440061987,
 0.44590738545536385,
 0.4787599229391603,
 0.42959666389696954,
 0.41767441882369405,
 0.4356173731529649,
 0.44075879231371523,
 0.4556643742133539,
 0.4594029684642264,
 0.44624971303551053,
 0.46142663490453883,
 0.49502356048612683,
 0.48714667790822846,
 0.44788617254159524,
 0.45793378915179467,
 0.47874422517782406,
 0.47900147265784915,
 0.47234933234958626,
 0.45551531076827395,
 0.4883760923980996,
 0.4873256804726167,
 0.4962763695630849,
 0.49016907

In [26]:
model_path = saved_path + f'/{identifier}.pt'
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint)
#model = load_model(model_name, model)
make_submission(identifier, model, test_loader)

Start prediction
DeepLabV3Plus(
  (encoder): RegNetEncoder(
    (stem): ConvBnAct(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (act): ReLU(inplace=True)
      )
    )
    (s1): RegStage(
      (b1): Bottleneck(
        (conv1): ConvBnAct(
          (conv): Conv2d(32, 168, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNormAct2d(
            168, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (act): ReLU(inplace=True)
          )
        )
        (conv2): ConvBnAct(
          (conv): Conv2d(168, 168, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
          (bn): BatchNormAct2d(
            168, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (act): ReLU(inplace=True)
          )
        )
        (conv3): ConvBnAct(
          (con

  0%|          | 0/105 [00:00<?, ?it/s]

End prediction.


In [26]:
model = smp.DeepLabV3Plus(
    encoder_name="timm-regnetx_064",
    encoder_weights="imagenet",
    in_channels=3,
    classes=N_CLASSES,
).to(device)
model_path = saved_path + f'/model_A.pt'
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint)
make_submission('model_A', model, test_loader)

Start prediction
DeepLabV3Plus(
  (encoder): RegNetEncoder(
    (stem): ConvBnAct(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (act): ReLU(inplace=True)
      )
    )
    (s1): RegStage(
      (b1): Bottleneck(
        (conv1): ConvBnAct(
          (conv): Conv2d(32, 168, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNormAct2d(
            168, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (act): ReLU(inplace=True)
          )
        )
        (conv2): ConvBnAct(
          (conv): Conv2d(168, 168, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
          (bn): BatchNormAct2d(
            168, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (act): ReLU(inplace=True)
          )
        )
        (conv3): ConvBnAct(
          (con

  0%|          | 0/105 [00:00<?, ?it/s]

End prediction.
